# Ichimoku Strategy
Source: https://www.investopedia.com/terms/i/ichimoku-cloud.asp


In [79]:

%run "Data and Graphs Definitions.ipynb"

In [49]:
'''
Used for recording buy triggers for tracking
'''
%run "Output To Trading Journal.ipynb"

In [50]:
import time
from datetime import datetime, timedelta


In [51]:
#Period defaulted to 9 so (9PH + 9PL)/2 by default
def TenkanValueForCurrentIndex(prices, index, period = 9):
    return (prices[index - period:index]['High'].max() + prices[index - period:index]['Low'].min()) / 2

#Period defaulted to 26 so (26PH + 26PL)/2 by default
def KijunValueForCurrentIndex(prices, index, period = 26):
    return (prices[index - period:index]['High'].max() + prices[index - period:index]['Low'].min()) / 2

In [52]:
'''
Adding Tenkan and Kijun data values
'''
def appendTenkanData(prices, period = 9):
    prices['Index'] = prices.reset_index().index + 1 #Care: Did not pass in inplace = true so original df still has the Time index (which is good)
    prices['Tenkan'] = prices['Index'].apply(lambda index: TenkanValueForCurrentIndex(prices, index, period))
    prices.drop(columns ="Index", inplace = True)
    return prices

def appendKijunData(prices, period = 26):
    prices['Index'] = prices.reset_index().index + 1 #Care: Did not pass in inplace = true so original df still has the Time index (which is good)
    prices['Kijun'] = prices['Index'].apply(lambda index: KijunValueForCurrentIndex(prices, index, period))
    prices.drop(columns ="Index", inplace = True)
    return prices

In [53]:
'''
Adds Ichimoku Tenkan and Kijun data and returns updated data frame
'''
def addTenkanKijunData(prices, tenkanPeriod = 9, kijunPeriod = 26):
    
    #Tenkan (Conversion Line)
    prices = appendTenkanData(prices, tenkanPeriod)
    
     #Kijun (Base Line)
    prices = appendKijunData(prices, kijunPeriod)
    
    #Don't need to dropNA values since graph just doesn't draw them so fine
    #Return updated dataframe
    return prices

In [54]:
'''
Add X periods to existing dataFrame (Note: this does not return an inplace update. Need to do oldDF = funcCall)
'''
def addNullFieldToData(prices, periods = 26):
    for i in range(0, periods - 1): #Do periods - 1 since Tradingview does not project last candle SSA,SSB since latest candle not complete
        secondLastEntry = pd.Series(prices.iloc[-2])
        lastEntry = pd.Series(prices.iloc[-1])
        timeDelta = lastEntry.name- secondLastEntry.name

        lastEntry.rename(prices.iloc[-1].name + timeDelta, inplace = True)
        entryToAppend = pd.DataFrame(data = [lastEntry])
        entryToAppend.iloc[-1] = 0 # 0 to represent NaN value

        prices =  prices.append(entryToAppend)

    return prices

In [55]:
'''
- Senkou Span A = Mid point of Tenkan and Kijun PLOTTED 26 Periods into the future
'''

def senkouSpanAForCurrentIndex(prices, index, period):
    return (prices.iloc[index - period]['Tenkan'] + prices.iloc[index - period]['Kijun']) / 2 

def appendSenkouSpanAData(prices, period):
    prices['Index'] = prices.reset_index().index + 1
    prices['SenkouA'] = prices['Index'].apply(lambda index: senkouSpanAForCurrentIndex(prices, index, period))
    prices.drop(columns ="Index", inplace = True)
    
    #Don't need to dropNA values since graph just doesn't draw them so fine
    
    return prices
    

  

In [56]:
'''
- Senkou Span B = 52 period mid point of highest and lowest prices PLOTTED 26 Periods into the future
'''
  
def sSBLengthPHPLMidPointForCurrentIndex(prices, index, ssbLength):
    return (prices[index - ssbLength:index]['High'].max() + prices[index - ssbLength:index]['Low'].min()) / 2


def senkouSpanBLengthPHPLMidPoint(prices, index, ssbLength):
    prices['SSBlengthPHPLMidPoint'] = prices['Index'].apply(lambda index: sSBLengthPHPLMidPointForCurrentIndex(prices, index, ssbLength))
    return prices
    
    
def senkouSpanBForCurrentIndex(prices, index, period, ssbLength):
    prices = senkouSpanBLengthPHPLMidPoint(prices, index, ssbLength)
    return prices.iloc[index - period]['SSBlengthPHPLMidPoint'] 

def appendSenkouSpanBData(prices, period, ssbLength):
    prices['Index'] = prices.reset_index().index + 1 
    prices['SenkouB'] = prices['Index'].apply(lambda index: senkouSpanBForCurrentIndex(prices, index, period, ssbLength))
    prices.drop(columns ="Index", inplace = True)
    
    #Don't need to dropNA values since graph just doesn't draw them so fine
    
    return prices
    


In [57]:
'''
Chikou (Lagging) Span data
'''

def chikouSpanForCurrentIndex(prices, index, period):
    if(index >= len(prices) - period + 1):
        return None
    else:
        return prices.iloc[index + period - 1]['Close']

def appendChikouSpan(prices, period):
    prices['Index'] = prices.reset_index().index
    prices['Chikou'] = prices['Index'].apply(lambda index: chikouSpanForCurrentIndex(prices, index, period))
    prices.drop(columns ="Index", inplace = True)
    return prices

# Note: 
Did not need to drop NaN values at any point since when plotting, it just ignores those values. The only issue was possibly when doing calculations with NaN values but I believe either those get ignored and set to NaN OR if the order of calculation is good, the calculations are done after data is available.

Indicating what part of the df to start at in the plotIchimoku function also ensured everything displays as it should. e.g. SSA and SSB are drawn from the data frame after the first senkouSpanPeriods input amount

In [58]:

def plotIchimoku(prices, tenkanPeriod = 9, kijunPeriod = 26, senkouSpanPeriods = 26, displacement = 52):
    #Necessary or when run this again on same df, that df has been altered since addTenkanKijunData() returns updated df
    prices = prices.copy()
    
    #Figure and adding Ichimoku data to df
    fig = drawCandleSticks(prices)
    
    prices = addTenkanKijunData(prices, tenkanPeriod, kijunPeriod)
    
    #Tenkan (Conversion Line)
    tenkanTrace =  go.Scatter(
                name = "Tenkan (Conversion Line)",
                mode = 'lines+markers',
                marker_symbol = 'circle',
                line=dict(color='blue', width=2),
                x= prices.index, 
                y= prices['Tenkan']
            )
    fig.add_trace(tenkanTrace)
    
     #Kijun (Base Line)
    kijunTrace =  go.Scatter(
                name = "Kijun (Base Line)",
                mode = 'lines+markers',
                marker_symbol = 'circle',
                line=dict(color='red', width=2),
                x= prices.index, 
                y= prices['Kijun']
            )
    fig.add_trace(kijunTrace)
    
    #Chikou Span (Lagging Span) -
    prices = appendChikouSpan(prices, senkouSpanPeriods)
    
    chikouSpanTrace =  go.Scatter(
                name = "Chikou Span",
                mode = 'lines+markers',
                marker_symbol = 'circle',
                line=dict(color='grey', width=2),
                x= prices.index, 
                y= prices['Chikou']
            )
    fig.add_trace(chikouSpanTrace)
    
    ##############################TRACES TO BE PLOTTED INTO FUTURE#######################################################
    
    #Now we add Extra fields for projected values (Adding before would cause errors since no candlesticks to draw for those)
    lastKnownPrice = prices.iloc[-1].name
    prices = addNullFieldToData(prices, senkouSpanPeriods)
    pricesKnown = prices[prices.index <= lastKnownPrice]
    pricesWithProjectionFields = prices[prices.index > lastKnownPrice]
    
    
    #Senkou Span A
    prices = appendSenkouSpanAData(prices, senkouSpanPeriods)
    senkouSpanATrace =  go.Scatter(
                name = "Senkou Span A",
                mode = 'lines+markers',
                marker_symbol = 'circle',
                line=dict(color='green', width=2),
                x= prices.iloc[senkouSpanPeriods:].index, 
                y= prices.iloc[senkouSpanPeriods:]['SenkouA']
            )
    fig.add_trace(senkouSpanATrace)
    
    
    #Senkou Span B
    prices = appendSenkouSpanBData(prices, senkouSpanPeriods, displacement)
    senkouSpanBTrace =  go.Scatter(
                name = "Senkou Span B",
                mode = 'lines+markers',
                marker_symbol = 'circle',
                line=dict(color='black', width=2),
                x= prices.iloc[senkouSpanPeriods:].index, 
                y= prices.iloc[senkouSpanPeriods:]['SenkouB']
            )
    fig.add_trace(senkouSpanBTrace)
    
    
    
    
    #---------------------------------LAYOUT------------------------------------------------#
    # Edit the layout (https://plotly.com/python/line-charts/)
    fig.update_layout(title='Ichimoku Cloud Setup',
                       xaxis_title= str(prices.index[1] - prices.index[0]) + " time frame",
                       yaxis_title= "Price")
    
    #Remove slider below since that was fixing y-axis to certain values even when zooming in
    fig.update_layout(xaxis_rangeslider_visible=False)
    return fig



In [59]:
'''
Output of Ichimoku data (used for calculation. The plotIchimoku() function is for visualization)

Note: First 26 (senkouSpanPeriods) are discarded. Given large enough data, this should not cause an issue.
'''

def ichimokuData(prices, tenkanPeriod = 9, kijunPeriod = 26, senkouSpanPeriods = 26, displacement = 52):
    prices = prices.copy()
    
    prices = addTenkanKijunData(prices, tenkanPeriod, kijunPeriod)
    
    #Chikou Span (Lagging Span) -
    prices = appendChikouSpan(prices, senkouSpanPeriods)
    
    
    lastKnownPrice = prices.iloc[-1].name
    prices = addNullFieldToData(prices, senkouSpanPeriods)
    pricesKnown = prices[prices.index <= lastKnownPrice]
    pricesWithProjectionFields = prices[prices.index > lastKnownPrice]
    
    prices = appendSenkouSpanAData(prices, senkouSpanPeriods)
    prices = appendSenkouSpanBData(prices, senkouSpanPeriods, displacement)
    
    #prices.iloc[:senkouSpanPeriods]['SenkouB'] = 'NULL'
   #prices.iloc[:senkouSpanPeriods]['SenkouA'] = 'NULL'
    
    prices = prices.iloc[senkouSpanPeriods:]
    prices = appendChikouSpan(prices, senkouSpanPeriods)
    return prices
    
    
    

In [60]:
'''
Adding Column for all Tenkan Kijun crosses
'''
def isTenkanGTkijun(prices, index):
    
    tenkan = prices.iloc[index]['Tenkan']
    kijun = prices.iloc[index]['Kijun']
    return tenkan > kijun
    

def didTKcrossLastCandle(prices, index):
    if(index == 0 or prices.iloc[index]['Open'] == 0):
        return False
    
    else:
        tenkanBelowAfterCross = isTenkanGTkijun(prices, index - 1) and not isTenkanGTkijun(prices, index)
        tenkanAboveAfterCross= not isTenkanGTkijun(prices, index - 1) and isTenkanGTkijun(prices, index)

        tkCrossed = tenkanBelowAfterCross or tenkanAboveAfterCross
        return tkCrossed


def addTKCrossesToData(prices):   
    prices['Index'] = prices.reset_index().index
    prices['didTKcross'] = prices['Index'].apply(lambda index : didTKcrossLastCandle(prices, index))
    prices.drop(columns ="Index", inplace = True)
    return prices


In [61]:

'''
Visualize cross locations 
'''

def plotTKCrossesTrace(fig, prices):
    crossesDF = prices[prices['didTKcross'] == True]
    for i in range(len(crossesDF)):
        fig.add_vline(x= crossesDF.index[i], line_width = 3, line_dash="dash", line_color="orange")
        
    return fig

In [62]:
usdtSymbols = getSymbolsList()
df = getCandlesData('FTMUSDT', "1w", "3 years")
ichiPlot = plotIchimoku(df, 9, 26, 26, 52) #Need to check spans. Seems like the dropNA values causing issues or the 0s at some point
ichiPlot
#TRY DRAWING JUST THE SPAN TRACES SEPARATE 

df = ichimokuData(df)
#df['SenkouA'].value_counts()
df = addTKCrossesToData(df)
#df


In [63]:
plotTKCrosses = plotTKCrossesTrace(ichiPlot, df)
plotTKCrosses

In [64]:
'''
Strategy (Long only with Ichimoku):

Note: Uses second last price to determine if buy condition is met since last candle is incomplete

Note: Try to do daily/weekly scans for condition being met on higher time frames (use those to determine)


---Look at this: https://www.youtube.com/watch?v=KE_SAzserLE&t=68s&ab_channel=TradePro

-----Seems like works well with forex and mostly because we see longer and thicker cloud.
    ------- Looks for lagging span to be above cloud, then current price tenkan > kijun and the projected cloud in the future 
                JUST turned green (i.e. ssA > ssB in latest projection)
    So look for long clouds (e.g. look at BTCUSD hourly, also good with those conditions)
    or e.g. SHOPIFY 4h 01 Nov 2021 another place all conditions met and general clouds are long so good to use strategy


Only on timeframes and charts where clouds keep one shape for a while (i.e. if red, stays red for long enough) 
Long: 
A) Lagging span above its cloud
B) Tenkan > Kijun
C) Price close above cloud (Currently not specifying if has to close above Red or Green cloud. Just above cloud.)
D) 26 candles later projected value of Senkou Span A > Senkou Span B

'''
def infoForDate(prices, date):
    return prices[prices.index == date]
    
def periodsTimeDelta(df, periods):
    return df.iloc[-1].name - df.iloc[-periods].name

def isLongBuyConditionMet(prices, buyDate):
    buyDateInfo = infoForDate(prices, buyDate)
    
    # Condition A
    info26CandlesAgo = infoForDate(prices, buyDate - periodsTimeDelta(prices, 26))
    #Using 26 candles ago to see if chikou above cloud
    pastChikou = info26CandlesAgo['Chikou'].values[0]
    pastSSA = info26CandlesAgo['SenkouA'].values[0]
    pastSSB = info26CandlesAgo['SenkouB'].values[0]
    conditionA = pastChikou > pastSSB and pastChikou > pastSSA  #chikou above ANY COLOUR CLOUD (ssb>ssa -> Red cloud)
    
    # Condition B
    conditionB = buyDateInfo['Tenkan'].values[0] > buyDateInfo['Kijun'].values[0]
    
    # Condition C
    closePrice = buyDateInfo['Close'].values[0]
    currentSSA = buyDateInfo['SenkouA'].values[0]
    currentSSB = buyDateInfo['SenkouB'].values[0]
    conditionC = closePrice > currentSSA and closePrice > currentSSB
    
    # Condition D
    info26CandlesLater = infoForDate(prices, buyDate + periodsTimeDelta(prices, 26))
    futureSSA = info26CandlesLater['SenkouA'].values[0]
    futureSSB = info26CandlesLater['SenkouB'].values[0]
    conditionD = futureSSA > futureSSB
    
    # All conditions true
    return conditionA and conditionB and conditionC and conditionD    


In [65]:
isLongBuyConditionMet(df, df[df['Open'] > 0].iloc[-2].name)

True

In [66]:
'''
Executing Ichimoku Strategy
-Returns true if price recently flipped to meeting all conditions
'''    
    
    
    
#Note: We can freely check how far back to look since we take the second last data point and strategy needs enough data regardless    
def executeBuy(prices):
    knownPrices = prices[prices['Open'] > 0]
    buyDate = knownPrices.iloc[-2].name
    
    #Note: periodsTimeDelta(df, 1) is 0, periodsTimeDelta(df, 2) is 1 candle
    if(isLongBuyConditionMet(prices, buyDate) and isLongBuyConditionMet(prices, buyDate - periodsTimeDelta(prices, 2)) == False):
        return True
    else:
        return False
        
        
    

In [67]:
executeBuy(df)

False

In [68]:
def getAllTickersToBuy(usdtSymbols, timeFrame, lookbackTime):
    tickersToBuy = []
    for ticker in usdtSymbols:
        df = getCandlesData(ticker, timeFrame, lookbackTime)
        #print("Working on " + ticker)
        if(len(df) < 150):
            continue
        else:
            df = ichimokuData(df)
            #df = addTKCrossesToData(df) (Commenting out for now. May be slowing current strategy since it doesnt use TK Crosses)
            if(executeBuy(df)):
                
                #Can only buy current candle price BUT executBuy(df) will look for trigger on previous candle
                buyThisCandle = df[df['Open'] > 0].iloc[-1]
                
                #Printing buy info
                print("Consider buying: -----> " + ticker + " | Current Candle: ")
                print(buyThisCandle)
                
                #Writing To Excel Output File Buy trigger
                writePositionToExcel("AlgoTradingJournal.xlsx", buyThisCandle, timeFrame)
                
                
                #Updating list of tickers to buy
                tickersToBuy.append(ticker)
    
    return tickersToBuy
            
            
            
        


In [69]:
def lookForPositionsToClose():
    fileName = "AlgoTradingJournal.xlsx"
    #Workbook
    wb = load_workbook(fileName)
    #Worksheet
    resultsWS = wb['test']

    for row in range(2, resultsWS.max_row + 1):
        if resultsWS.cell(row, 8).value == "Active":

            ticker = resultsWS.cell(row, 3).value
            
            #Exit prices
            targetPrice = resultsWS.cell(row, 6).value
            stopLossPrice = resultsWS.cell(row, 7).value

            #Get Latest Data to see if can close

            latestMinuteData = getCandlesData(ticker, '1m', '1m')
            currentClose = latestMinuteData.iloc[-1]['Close']

            isExitPosition = currentClose <= stopLossPrice or currentClose >= targetPrice

            if(isExitPosition and currentClose > 0):
                timeAtExit = latestMinuteData.iloc[-1].name
                
                entryPrice = resultsWS.cell(row, 4).value
                profitPct = (currentClose / entryPrice) - 1
                
                print("Exiting: " + str(ticker) + " at price of: " + str(currentClose) + " details:\n " + str(latestMinuteData))
                
                #Column H is Status
                resultsWS.cell(row, 8, "Closed")

                #Column I is Price at Exit
                resultsWS.cell(row, 9, currentClose)

                #Column J is Time at Exit
                resultsWS.cell(row, 10, timeAtExit)
                
                #Column K is Profit %
                resultsWS.cell(row, 11, profitPct)

    #Saving workbook
    wb.save(fileName)

        

In [70]:
def whileWaitingUntilNextAvailableCandle(secondsPassedSoFar, timeFrame):
    timeInMinutesDict = {'5m': 5, '15m': 15, '30m': 30, '1h': 60, '4h': 240}
    timeInSeconds = 60 * timeInMinutesDict[timeFrame]   
    sleepTime = 60
        
    searchTimeSeconds = timeInSeconds - secondsPassedSoFar
    
    start = datetime.now()
    end = start + timedelta(seconds = searchTimeSeconds)
    now = datetime.now()
    while now < end:
        #Search for Active positions in excel file that can be closed
        lookForPositionsToClose()
        time.sleep(sleepTime)
        now = datetime.now()

In [71]:

def getBuyListEveryTimeFrame(symbolsToLookAt, timeFrame, hoursDuration = 0.5):
    buyTickersDictionary = []
    startTime = datetime.now()
    endTime= startTime + timedelta(hours = hoursDuration)
    now = datetime.now() # current date and time
    while now < endTime:
        now = datetime.now()
        

        #Searching for tickers to buy
        
        timeLookBackDict = {'5m': "1 days", '15m':"2 days", '30m':"4 days", '1h': "1 week", '4h': "1 month" }
        print("Looking for buy opportunities at: " + str(now))
        tickersToBuy = getAllTickersToBuy(symbolsToLookAt, timeFrame, timeLookBackDict[timeFrame])
        
        
        #Time Passed So Far
        
        timePassedSoFar = datetime.now() - now
        secondsPassedSoFar = timePassedSoFar.seconds
        
        
        #Updating dictionary with time and list returned
        
        buyTickersDictionary.append({str(datetime.now()):tickersToBuy})
        print("Finished looking at: " + str(datetime.now()))
        
        
        #Waiting before looping again
        
        print("Tickers to buy from current search:" + str(tickersToBuy))
        print("Checking to close other trades while waiting for next candles.")
        
        #Instead of sleeping, looks to close active positions while waiting for next candles
        whileWaitingUntilNextAvailableCandle(secondsPassedSoFar, timeFrame)
        
        #time.sleep(60 * 5) # Delay for 1 minute (60 seconds) * 5 = 5min.
        
    return buyTickersDictionary

In [72]:
lowerTFsymbols = ['BTCUSDT','ETHUSDT','LINKUSDT','FTMUSDT','ALGOUSDT','HBARUSDT','LRCUSDT']

higherTFsymbols = ['BTCUSDT','ETHUSDT','LINKUSDT','FTMUSDT','ALGOUSDT','HBARUSDT','LRCUSDT', 
                   'NEARUSDT','FILUSDT', 'ALICEUSDT', 'QNTUSDT', 'SPELLUSDT', 'KP3RUSDT', 'DYDXUSDT', 'CAKEUSDT',
                  '1INCHUSDT','AVAXUSDT']


In [78]:
#allListsHTF = getBuyListEveryTimeFrame(higherTFsymbols, '1h', 8)
allListsLTF = getBuyListEveryTimeFrame(lowerTFsymbols, '15m', 1)

Looking for buy opportunities at: 2022-01-16 17:31:18.831251
Consider buying: -----> FTMUSDT | Current Candle: 
Open                      3.2731
High                      3.2741
Low                       3.2567
Close                     3.2625
Volume                     78275
Ticker                   FTMUSDT
Tenkan                    3.2607
Kijun                    3.25485
Chikou                         0
SenkouA                  3.24835
SSBlengthPHPLMidPoint     3.2551
SenkouB                   3.1351
Name: 2022-01-16 22:30:00, dtype: object
Consider buying: -----> ALGOUSDT | Current Candle: 
Open                       1.4372
High                       1.4378
Low                         1.437
Close                      1.4371
Volume                       7607
Ticker                   ALGOUSDT
Tenkan                     1.4287
Kijun                       1.427
Chikou                          0
SenkouA                   1.40492
SSBlengthPHPLMidPoint      1.4094
SenkouB                  

In [ ]:
allsymbs = getSymbolsList()
tickersToBuy = getAllTickersToBuy(allsymbs,"4h", "1 month")